In [1]:
import gensim
import numpy as np
import time
from datetime import timedelta

model = gensim.models.Word2Vec.load('spamDetect_20180217.model')

In [2]:
import pandas as pd
import random

# train = pd.read_csv( "shuf_spam_train.csv", header=0, delimiter=",", quoting=2)
train = pd.read_csv( "spam_train_5_5.csv")
print("Read %d lines..." % (train["sentence"].shape))

train_data = train["sentence"]
train_label = train["flag"]


Read 8019 lines...


In [3]:
num_features = 324

def makeFeatureVec(words, model, num_features):

    featureVec = np.zeros((num_features,),dtype="float32")

    nwords = 0

    index2word_set = set(model.wv.index2word)

    for word in words:
        if word.decode("utf-8") in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word.decode("utf-8")])
#             print("========",word,":",model[word.decode("utf-8")])
    if(nwords > 0):
        featureVec = np.divide(featureVec,nwords)
        
#     print(featureVec)
    return featureVec

def getAvgFeatureVecs(sentences, model, num_features):
    
    counter = 0
    print(len(sentences))
    reviewFeatureVecs = np.zeros((len(sentences),num_features),dtype="float32")

    for sentence in sentences:
       if(isinstance(sentence,basestring)==False):
            sentence=" "
       if counter%1000 == 0:
           print "Review %d of %d" % (counter, len(sentences))
       words = sentence.split(" ")
       reviewFeatureVecs[counter] = makeFeatureVec(words, model, num_features)

       counter = counter + 1
    return reviewFeatureVecs

In [4]:
train_clean_data = []
for review in train["sentence"]:
    train_clean_data.append(review)
# print(type(train_clean_data)) 
# print(train_clean_data[0])
train_clean_data = getAvgFeatureVecs(train_clean_data, model, num_features)
# print(type(train_clean_data))
# print(len(train_clean_data))
# print(train_clean_data)

8019
Review 0 of 8019


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Review 1000 of 8019
Review 2000 of 8019
Review 3000 of 8019
Review 4000 of 8019
Review 5000 of 8019
Review 6000 of 8019
Review 7000 of 8019
Review 8000 of 8019


In [5]:
print(train_clean_data[1])
print(train_data[1])
print(model[u'绿茶'])

[-0.13448544 -0.15526298  0.27640295  0.24393681 -0.031809    0.47941011
 -0.22217706  0.06064174 -0.20854169 -0.25044388 -0.07815401 -0.36584005
  0.13301717 -0.33904946  0.06741552  0.153918   -0.07450511 -0.00255099
 -0.0229926   0.01189698  0.13944772 -0.16746029 -0.12637469 -0.34101805
 -0.16907683 -0.22200537 -0.00183525  0.11370945 -0.06828809 -0.03814153
 -0.02239798  0.16260451 -0.13899696  0.02925395 -0.07704003  0.00144653
 -0.04167575  0.12710071  0.06970742 -0.04279945 -0.1034698  -0.01272895
 -0.07456801  0.03568219 -0.1139776   0.4366076   0.13906632 -0.07055129
 -0.06471727  0.14301565 -0.20270628 -0.01353151 -0.28606248  0.11344427
 -0.05270681 -0.07796086  0.11920242 -0.2101891   0.07080494 -0.03920368
 -0.14051589 -0.12718305  0.24923721  0.2017079   0.19455475 -0.22270826
  0.00910456  0.01823404  0.0525796   0.21754676 -0.16793996  0.08711456
  0.07909136 -0.14023148 -0.02585023  0.29666746  0.16141331 -0.19616255
  0.01133293 -0.01034601  0.23769563  0.26301605 -0

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
import tensorflow as tf
from tensorflow.contrib import rnn

lr = 1e-3
# 在训练和测试的时候，我们想用不同的 batch_size.所以采用占位符的方式
batch_size = tf.placeholder(tf.int32,[])  # 注意类型必须为 tf.int32
# 在 1.0 版本以后请使用 ：
# keep_prob = tf.placeholder(tf.float32, [])
# batch_size = tf.placeholder(tf.int32, [])

# 每个时刻的输入特征是28维的，就是每个时刻输入一行，一行有 28 个像素
input_size = 324
# 时序持续长度为28，即每做一次预测，需要先输入28行
timestep_size = 1
# 每个隐含层的节点数
hidden_size = 324
# LSTM layer 的层数
layer_num = 2
# 最后输出分类类别数量，如果是回归预测的话应该是 1
class_num = 2

_X = tf.placeholder(tf.float32, [None, input_size])
y = tf.placeholder(tf.float32, [None,class_num])
keep_prob = tf.placeholder(tf.float32)

In [7]:
# 把784个点的字符信息还原成 28 * 28 的图片
# 下面几个步骤是实现 RNN / LSTM 的关键
####################################################################
# **步骤1：RNN 的输入shape = (batch_size, timestep_size, input_size) 
X = tf.reshape(_X, [-1, timestep_size,input_size])

# **步骤2：定义一层 LSTM_cell，只需要说明 hidden_size, 它会自动匹配输入的 X 的维度
lstm_cell = rnn.BasicLSTMCell(num_units=hidden_size, forget_bias=1.0, state_is_tuple=True)

# **步骤3：添加 dropout layer, 一般只设置 output_keep_prob
lstm_cell = rnn.DropoutWrapper(cell=lstm_cell, input_keep_prob=1.0, output_keep_prob=keep_prob)

# **步骤4：调用 MultiRNNCell 来实现多层 LSTM
mlstm_cell = rnn.MultiRNNCell([lstm_cell] * layer_num, state_is_tuple=True)

# **步骤5：用全零来初始化state
init_state = mlstm_cell.zero_state(batch_size, dtype=tf.float32)

# **步骤6：方法一，调用 dynamic_rnn() 来让我们构建好的网络运行起来
# ** 当 time_major==False 时， outputs.shape = [batch_size, timestep_size, hidden_size] 
# ** 所以，可以取 h_state = outputs[:, -1, :] 作为最后输出
# ** state.shape = [layer_num, 2, batch_size, hidden_size], 
# ** 或者，可以取 h_state = state[-1][1] 作为最后输出
# ** 最后输出维度是 [batch_size, hidden_size]
outputs, state = tf.nn.dynamic_rnn(mlstm_cell, inputs=X, initial_state=init_state, time_major=False)
h_state = outputs[:, -1, :]  # 或者 h_state = state[-1][1]

# *************** 为了更好的理解 LSTM 工作原理，我们把上面 步骤6 中的函数自己来实现 ***************
# 通过查看文档你会发现， RNNCell 都提供了一个 __call__()函数（见最后附），我们可以用它来展开实现LSTM按时间步迭代。
# **步骤6：方法二，按时间步展开计算
# outputs = list()
# state = init_state
# with tf.variable_scope('RNN'):
#     for timestep in range(timestep_size):
#         if timestep > 0:
#             tf.get_variable_scope().reuse_variables()
#         # 这里的state保存了每一层 LSTM 的状态
#         (cell_output, state) = mlstm_cell(X[:, timestep, :], state)
#         outputs.append(cell_output)
# h_state = outputs[-1]

In [8]:
# 上面 LSTM 部分的输出会是一个 [hidden_size] 的tensor，我们要分类的话，还需要接一个 softmax 层
# 首先定义 softmax 的连接权重矩阵和偏置
# out_W = tf.placeholder(tf.float32, [hidden_size, class_num], name='out_Weights')
# out_bias = tf.placeholder(tf.float32, [class_num], name='out_bias')
# 开始训练和测试
W = tf.Variable(tf.truncated_normal([hidden_size, class_num], stddev=0.1), dtype=tf.float32)
bias = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
y_pre = tf.nn.softmax(tf.matmul(h_state, W) + bias)
y_pred_cls = tf.argmax(y_pre, 1)

# 损失和评估函数
cross_entropy = -tf.reduce_mean(y * tf.log(y_pre))
train_op = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [9]:
#初始化sess
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)

In [10]:
import os
from sklearn.preprocessing import OneHotEncoder
    
num_iterations = len(train_clean_data) - 128   

if(os.path.exists("/notebooks/SpamTrain/model_rnn_03/checkpoint") == False): 
    print("begin to train...")
    
    #处理标签数据
    label = train["flag"]
    train_data = train_clean_data[:,:]
    train_label = label[:]
    
    # 使用ont-hot编码
    enc = OneHotEncoder().fit(train_label.reshape(-1, 1))
    train_label = enc.transform(train_label.reshape(-1, 1)).toarray()


    for i in range(num_iterations):
        _batch_size = 128
        
        offset = (i * _batch_size) % (int(len(train_clean_data) * 1) - _batch_size)

        batch_x = train_data[offset:(offset + _batch_size),:]
        batch_y = train_label[offset:(offset + _batch_size),:]

        if (i+1)%100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={
                _X:batch_x, y: batch_y, keep_prob: 1.0, batch_size: _batch_size})
            # 已经迭代完成的 epoch 数: mnist.train.epochs_completed
            print " step %d, training accuracy %g" % ((i+1), train_accuracy)
        sess.run(train_op, feed_dict={_X: batch_x, y: batch_y, keep_prob: 0.5, batch_size: _batch_size})
        
    saver_path = saver.save(sess, "/notebooks/SpamTrain/model_rnn_03/comment_rnn_model.ckpt")  # 保存模型
else:
   print("restore the model...")
   saver.restore(sess,'/notebooks/SpamTrain/model_rnn_03/comment_rnn_model.ckpt') 

restore the model...
INFO:tensorflow:Restoring parameters from /notebooks/SpamTrain/model_rnn_03/comment_rnn_model.ckpt


In [11]:
batch_x = train_clean_data[0:1000,:]
print(batch_x.shape)
print(train_clean_data.shape)
pre = sess.run(y_pred_cls, {_X:batch_x, keep_prob: 1.0,batch_size: 1000})

# print(type(train_clean_data))
# print(train_clean_data)
# print(2053976 / 256)
print(batch_x)

nwords = 0
for i in range(len(pre)):
    if(pre[i] == 1):
        print(train_data[i])
#         train_clean_data = np.append(train_clean_data,train_clean_data[i])
        nwords += 1

# train_clean_data = train_clean_data.reshape(-1,2053976 / 256,256)
print(train_clean_data.shape)
# print(train_clean_data.shape[0] / 256)
# print(train_clean_data)

print(nwords)

(1000, 324)
(8019, 324)
[[-0.05196826  0.62578809  0.33745331 ...,  0.07365704 -0.18128303
   0.05877255]
 [-0.13448544 -0.15526298  0.27640295 ..., -0.28622359 -0.1406382
  -0.25426164]
 [-0.04657002  0.63925362 -0.04474411 ..., -0.41634336 -0.48093891
   0.03007146]
 ..., 
 [-0.45400429 -0.66608202  0.32898128 ..., -0.7339204   0.10969409
  -0.62974346]
 [-0.09312254  0.05677414  0.81271517 ..., -0.12547359 -0.39977166
   0.511648  ]
 [-0.52973425  0.04141427  0.20068628 ..., -0.30700812 -0.38732147
   0.33269024]]
趟 单位 送 钥匙 加微 信聊
男主角 大戏 精在 折腾 够呛 平淡 适合 实在 学生妹 法眼 没什么 地方 搭调
回复 aa1180 历史 课本上 读到 东西 真 理 这头 蠢货 真的 屠宰场 日本 鬼 子 李向阳 跑 解放 战 争中 苏联 缴获 日本 武器 我军 头脑 太 简单 胜利 想 太 笫 一应 补脑 应 补钙 软骨 狗
手 残挡
优秀 子 非 同学 昨天 说 发 请 名字 呼唤 链接 推荐 公众 号 资源 很全
嗨 解决 咨询 教 提升 能力 执行力 构建 思维 体系 相处 提高 情商 提升 沟通 水平 吸引 异性 获取 异性 亲睐 面对 恋爱 关系 克服 心理障碍 免费 答疑 解惑 恋爱 心态 社交 障碍 人际交往 情商 思维 做事 方法 免费 加群 学习 资料 请加 QQ380125457 希望 成长
红外线 针孔 摄像头
绿色 生态园 一点 包括 同伴 间 绿色 和谐 关系 潇潇 老师 有意识 幼儿园 发展 理念 落实 幼儿 日常 习惯 培养 正面 角度 引导 幼幼间 绿色 关系 建立 孩子 绿色 氛围 生活 阳光 快乐
